In [61]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [62]:
out='../output/'
inb='../input/flow/'
pref='idm'
inp = inb + pref + '/'

In [63]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-01-01,160,ZAMZ,0,ZAMC
1,2016-01-01,170,ZIM,0,ZAMZ
2,2016-01-01,160,ZAMZ,1,ZAMC
3,2016-01-01,170,ZIM,1,ZAMZ
4,2016-01-01,160,ZAMZ,2,ZAMC
...,...,...,...,...,...
6510,2023-07-31,51,MOZS,23,ESW
6511,2023-07-31,69,ZAMZ,23,ZAMC
6512,2023-07-31,11,ZAMZ,23,ZAML
6513,2023-07-31,80,ZIM,23,ZAMZ


In [64]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

C:\Users\kabom\AppData\Local\Temp\ipykernel_27836\1849715270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dam=dam.drop(['date'],1)


price  volume  year  month  day  day_of_week      value  \
date       hour                                                            
2016-01-01 0     40.10   477.5  2016      1    1            4  19147.750   
           1     40.08   486.6  2016      1    1            4  19502.928   
           2     40.06   495.6  2016      1    1            4  19853.736   
           3     40.08   496.6  2016      1    1            4  19903.728   
           4     40.07   502.9  2016      1    1            4  20151.203   
...                ...     ...   ...    ...  ...          ...        ...   
2023-07-31 19    95.07    71.0  2023      7   31            0   6749.970   
           20    27.32   243.0  2023      7   31            0   6638.760   
           21    23.90   145.6  2023      7   31            0   3479.840   
           22    21.93   373.0  2023      7   31            0   8179.890   
           23    17.99   431.0  2023      7   31            0   7753.690   

                      cat  
date       hour            
2016-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2023-07-31 19     evening  
           20    standard  
           21    standard  
           22         off  
           23         off  

[65572 rows x 8 columns]

In [65]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

C:\Users\kabom\AppData\Local\Temp\ipykernel_27836\2288514685.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  flow=flow.dropna(0)


,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.00
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.00
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.80
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.60
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.60
...,...,...,...,...,...,...,...,...,...,...,...
486465,2023-07-31,23,51,MOZS,ESW,2023,7,31,0,17.99,917.49
486466,2023-07-31,23,69,ZAMZ,ZAMC,2023,7,31,0,17.99,1241.31
486467,2023-07-31,23,11,ZAMZ,ZAML,2023,7,31,0,17.99,197.89
486468,2023-07-31,23,80,ZIM,ZAMZ,2023,7,31,0,17.99,1439.20


In [66]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.00,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.00,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.80,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.60,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.60,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
486465,2023-07-31,23,-51,-917.49,17.99,MOZS,2023,7,31,0
486466,2023-07-31,23,-69,-1241.31,17.99,ZAMZ,2023,7,31,0
486467,2023-07-31,23,-11,-197.89,17.99,ZAMZ,2023,7,31,0
486468,2023-07-31,23,-80,-1439.20,17.99,ZIM,2023,7,31,0


In [67]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [68]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week    index  flow    value  \
date       hour                                                                
2016-01-01 0     ZAMC  2016      1    1            4        0   160  6416.00   
           0     ZAMZ  2016      1    1            4        1    10   401.00   
           0      ZIM  2016      1    1            4        1  -170 -6817.00   
           1     ZAMC  2016      1    1            4        2   160  6412.80   
           1     ZAMZ  2016      1    1            4        5    10   400.80   
...               ...   ...    ...  ...          ...      ...   ...      ...   
2023-07-31 23     MOZ  2023      7   31            0  1459392  -216 -3885.84   
           23     RSA  2023      7   31            0   486464   100  1799.00   
           23    ZAMC  2023      7   31            0   486466    69  1241.31   
           23    ZAML  2023      7   31            0   486467    11   197.89   
           23     ZIM  2023      7   31            0  2432331   200  3598.00   

                 price  cat  
date       hour              
2016-01-01 0     40.10  off  
           0     80.20  off  
           0     40.10  off  
           1     40.08  off  
           1     80.16  off  
...                ...  ...  
2023-07-31 23    53.97  off  
           23    17.99  off  
           23    17.99  off  
           23    17.99  off  
           23    89.95  off  

[278077 rows x 10 columns]

In [69]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum       value  hour_count  \
year month node cat      side                                        
2016 1     BOT  evening  supply        -25    -1501.40           2   
                morning  supply       -285   -14572.40           4   
                off      supply      -1270   -59474.14          45   
                standard supply      -1165   -57809.40          18   
           HCB  evening  supply        -57    -6204.59           2   
...                                    ...         ...         ...   
2023 7     ZIM  morning  consume         1       34.98           1   
                         supply         -1      -34.97           1   
                off      consume     42497  1078278.34         283   
                standard consume      3400    75192.90          44   
                         supply         -2      -71.88           2   

                                  price_weight_mean  price_mean       power  
year month node cat      side                                                
2016 1     BOT  evening  supply           60.056000   60.050000  -12.500000  
                morning  supply           51.131228   55.290000  -71.250000  
                off      supply           46.830031   48.463556  -28.222222  
                standard supply           49.621803   56.768333  -64.722222  
           HCB  evening  supply          108.852456   99.140000  -28.500000  
...                                             ...         ...         ...  
2023 7     ZIM  morning  consume          34.980000  174.900000    1.000000  
                         supply           34.970000  174.850000   -1.000000  
                off      consume          25.373046  133.675972  150.166078  
                standard consume          22.115559  109.662045   77.272727  
                         supply           35.940000  179.700000   -1.000000  

[3505 rows x 6 columns]

In [70]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')